In [1]:
import pandas as pd
import numpy as np
import glob
import datetime
import os

In [2]:
TMR_up_to_2019Q3=pd.read_csv("/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2019Q3/output/BL_MMM_long_cumu_up_to_2019Q3_JL_2020-03-06.csv",dtype=str)
TMR_up_to_2019Q3['impression']=TMR_up_to_2019Q3['impression'].astype(float).astype(int)
TMR_up_to_2019Q3['click']=TMR_up_to_2019Q3['click'].astype(float).astype(int)
TMR_up_to_2019Q3['cost']=TMR_up_to_2019Q3['cost'].astype(float)


In [3]:
TMR_up_to_2019Q3_no_DMA=TMR_up_to_2019Q3.groupby(["week date","media","submedia","placement"])['impression','click','cost'].sum().reset_index()
print(TMR_up_to_2019Q3_no_DMA['week date'].min())
print(TMR_up_to_2019Q3_no_DMA['week date'].max())
# TMR_up_to_2019Q3_no_DMA

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


2016-09-25
2019-10-27


In [4]:
df_pandora=TMR_up_to_2019Q3_no_DMA[TMR_up_to_2019Q3_no_DMA['submedia'].apply(lambda x: "pandora" in x.lower())]
# sorted(df_pandora['week date'].unique().tolist())

In [5]:
qc_date_start="2019-05-05"
qc_date_end="2019-08-03"

qc_df=TMR_up_to_2019Q3_no_DMA[TMR_up_to_2019Q3_no_DMA['week date']>=qc_date_start]
qc_df=qc_df[qc_df['week date']<=qc_date_end]

qc_df['media_submedia']=qc_df['media']+"|"+qc_df['submedia']
print(sorted(qc_df['media_submedia'].unique().tolist()))

# QC week date as 
qc_df['week_as_date'] = pd.to_datetime(qc_df['week date'])
qc_df['day_of_week'] = qc_df['week_as_date'].dt.day_name()
print(qc_df['day_of_week'].unique())
del qc_df['week_as_date']
del qc_df['day_of_week']

['Circulation|xx', 'Digital|Flipp', 'Digital|PLA', 'Digital|Pandora', 'Digital|Programmatic', 'Digital|SEM', 'Digital|Social', 'Digital|YouTube', 'Email|xx', 'TV|National']
['Sunday']


In [6]:
print(sorted(qc_df['media'].unique().tolist()))

['Circulation', 'Digital', 'Email', 'TV']


In [7]:
print(sorted(qc_df['submedia'].unique().tolist()))

['Flipp', 'National', 'PLA', 'Pandora', 'Programmatic', 'SEM', 'Social', 'YouTube', 'xx']


In [8]:
qc_media="Circulation"
qc_submedia="xx"

print(qc_media,qc_submedia)
for col in ['impression','click','cost']:
    df=qc_df[((qc_df['media']==qc_media) & (qc_df['submedia']==qc_submedia))]
    print(col,qc_df[qc_df['media']==qc_media][col].sum())

Circulation xx
impression 82346551
click 0
cost 4706007.0


In [9]:
list_datorama_files=glob.glob("./datorama_jian_pull_2019Q2/*.csv")
list_datorama_files.sort()
list_datorama_files

['./datorama_jian_pull_2019Q2/Digital Weekly Media Data  (Search).csv',
 './datorama_jian_pull_2019Q2/Digital Weekly Media Data without Placement (excluded FB).csv',
 './datorama_jian_pull_2019Q2/E-circular_Flipp & Print Circular & Email by Week.csv',
 './datorama_jian_pull_2019Q2/Social Daily Media Data.csv']

In [10]:
df_1_search=pd.read_csv(list_datorama_files[0])
df_1_search.columns=[x.lower() for x in df_1_search.columns.tolist()]
df_1_search=df_1_search.rename(columns={"impressions":"impression","clicks":"click","media cost":"cost"})
df_1_search=df_1_search[['week','media','submedia','placement','impression','click','cost']]
df_1_search['submedia']=df_1_search['submedia'].replace({"Paid SEM Campaign":"SEM"}).replace({"Shopping Campaign":"PLA"})
df_1_search=df_1_search.sort_values(['media','submedia','placement','week'])
df_1_search.head(2)

,week,media,submedia,placement,impression,click,cost
3,2019-05-05 00:00,Digital,PLA,Bing,908500.0,10257.0,3954.33
5,2019-05-12 00:00,Digital,PLA,Bing,45061.0,707.0,1108.43


In [11]:
df_2_digital=pd.read_csv(list_datorama_files[1])
df_2_digital.columns=[x.lower() for x in df_2_digital.columns.tolist()]
df_2_digital=df_2_digital.rename(columns={"impressions":"impression","clicks":"click","media cost":"cost"})
df_2_digital=df_2_digital[['week','media','submedia','impression','click','cost','partner','tactic']]

df_2_digital_YT=df_2_digital[df_2_digital['partner']=="YouTube"]
df_2_digital_YT['placement']=df_2_digital_YT['submedia']
df_2_digital_YT['submedia']="YouTube"
df_2_digital_YT=df_2_digital_YT[df_1_search.columns.tolist()]

df_2_digital_Pandora=df_2_digital[df_2_digital['partner']=="Pandora"]
df_2_digital_Pandora['placement']=df_2_digital_Pandora['submedia']
df_2_digital_Pandora['submedia']="Pandora"
df_2_digital_Pandora=df_2_digital_Pandora[df_1_search.columns.tolist()]

df_2_digital_Programmatic=df_2_digital[~df_2_digital['partner'].isin(["YouTube",'Pandora'])]
df_2_digital_Programmatic['placement']=df_2_digital_Programmatic['submedia']
df_2_digital_Programmatic['submedia']="Programmatic"
df_2_digital_Programmatic=df_2_digital_Programmatic[df_1_search.columns.tolist()]

df_2_digital=df_2_digital_YT.append(df_2_digital_Pandora).append(df_2_digital_Programmatic)


df_2_digital.head(2)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

,week,media,submedia,placement,impression,click,cost
1,2019-07-28 00:00,Digital,YouTube,Multiplatform Video,0.0,0.0,117.782331
8,2019-07-28 00:00,Digital,YouTube,Multiplatform Video,2264114.0,4309.0,14035.714747


In [12]:
df_3_email_ecirc=pd.read_csv(list_datorama_files[2])
df_3_email_ecirc.columns=[x.lower() for x in df_3_email_ecirc.columns.tolist()]
df_3_email_ecirc=df_3_email_ecirc.rename(columns={"impressions":"impression","clicks":"click","media cost":"cost"})
df_3_email_ecirc=df_3_email_ecirc[['week','media','placement','impression','click','cost']]
df_3_email_ecirc.insert(2,"submedia",np.nan)
df_3_email_ecirc['submedia']=np.where(df_3_email_ecirc['media']=="Digital","Flipp","xx")
df_3_email_ecirc['placement']=np.where(df_3_email_ecirc['media']=="Digital",df_3_email_ecirc['placement'],"xx")

df_3_email_ecirc.head(3)

,week,media,submedia,placement,impression,click,cost
0,2019-07-28 00:00,Digital,Flipp,Hosted,172398.0,148604.0,0.00
1,2019-07-28 00:00,Digital,Flipp,Native Network,8.0,5.0,1.45
2,2019-07-28 00:00,Digital,Flipp,Flipp App,144125.0,70875.0,20553.75


In [13]:
df_4_email_social=pd.read_csv(list_datorama_files[3])
df_4_email_social.columns=[x.lower() for x in df_4_email_social.columns.tolist()]
df_4_email_social=df_4_email_social.rename(columns={"impressions":"impression","clicks":"click","media cost":"cost"})

df_4_email_social=df_4_email_social[['week','media','submedia','partner','impression','click','cost']]
df_4_email_social=df_4_email_social.rename(columns={"partner":"placement"})
df_4_email_social.head(3)

,week,media,submedia,placement,impression,click,cost
0,2019-05-05 00:00,Digital,Social,Pinterest,1481491.0,7062.0,7850.78
1,2019-05-05 00:00,Digital,Social,Pinterest,1932672.0,9185.0,9508.40
2,2019-05-05 00:00,Digital,Social,Facebook,3182642.0,18827.0,12685.75


In [14]:
df_all_digital_from_datorama=pd.concat([df_1_search,df_2_digital,df_3_email_ecirc,df_4_email_social])
df_all_digital_from_datorama['week']=df_all_digital_from_datorama['week'].apply(lambda x: x.split(" ")[0])
df_all_digital_from_datorama=df_all_digital_from_datorama.rename(columns={"week":"week date"})

# QC week date as 
df_all_digital_from_datorama['week_as_date'] = pd.to_datetime(df_all_digital_from_datorama['week date'])
df_all_digital_from_datorama['day_of_week'] = df_all_digital_from_datorama['week_as_date'].dt.day_name()
print(df_all_digital_from_datorama['day_of_week'].unique())
del df_all_digital_from_datorama['week_as_date']
del df_all_digital_from_datorama['day_of_week']

df_all_digital_from_datorama.head(2)

['Sunday']


,week date,media,submedia,placement,impression,click,cost
3,2019-05-05,Digital,PLA,Bing,908500.0,10257.0,3954.33
5,2019-05-12,Digital,PLA,Bing,45061.0,707.0,1108.43


In [15]:
df_all_digital_from_datorama['week date'].min(),df_all_digital_from_datorama['week date'].max()

('2019-05-05', '2019-07-28')

In [27]:
# Looks fine
# Ecirc and the Email cost were adjusted later

for comb,df_group in df_all_digital_from_datorama.groupby(['media','submedia']):
    media=comb[0]
    submedia=comb[1]
    
    impr=df_group['impression'].sum()
    click=df_group['click'].sum()
    cost=df_group['cost'].sum()
    
    qc_xx=qc_df[(qc_df['media']==media) & (qc_df['submedia']==submedia)]
    qc_impr=qc_xx['impression'].sum()
    qc_click=qc_xx['click'].sum()
    qc_cost=qc_xx['cost'].sum()
    
    print(comb)
    print("impr",np.round(impr/qc_impr,4),impr,qc_impr)
    print("click",np.round(click/qc_click,4),click,qc_click)
    print("cost",np.round(cost/qc_cost,4),cost,qc_cost)
    

('Digital', 'Flipp')
impr 0.9998 5695713.0 5696888
click 1.0982 4075995.0 3711617
cost 1.0113 360191.8900000001 356157.99999999994
('Digital', 'PLA')
impr 1.0 114693720.0 114693720
click 1.0 1983612.0 1983612
cost 1.0 273027.64999999997 273027.64999999915
('Digital', 'Pandora')
impr 1.0 6435048.0 6435048
click 1.0 5568.0 5568
cost 1.0 128700.95999999999 128700.95999999985
('Digital', 'Programmatic')
impr 1.0 127877927.0 127877927
click 1.0 587828.0 587828
cost 1.0 905123.8324714281 905123.8324714267
('Digital', 'SEM')
impr 1.0 17750475.0 17750475
click 1.0 4279300.0 4279300
cost 1.0 871843.3700079999 871843.3700079994
('Digital', 'Social')
impr 1.0 235474428.0 235474428
click 1.0 1766639.0 1766639
cost 1.0 1339745.2119049996 1339745.2119049972
('Digital', 'YouTube')
impr 1.0 36931667.0 36931667
click 1.0 92546.0 92546
cost 1.0 271408.260334 271408.2603339994
('Email', 'xx')
impr 1.0 2033659710.0 2033659691
click 1.0007 16139726.0 16128461
cost 1.7466 926910.6117754851 530690.0
